In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F

In [ ]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [ ]:
!pip install transformers

In [ ]:
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)

In [ ]:
# Check library version
#!pip list | grep -E 'transformers|torch|Keras'

This notebook work with env:

- Keras                2.3.1                 
- torch                1.1.0                 
- transformers         2.2.0      

# Introduction

In this notebook, will introduce how to do text classification with XLNet, including:

- Load and preprocess data
- Parser data
- Make training data
- Train model
- Evaluate result

Tips:
 - Update PyTorch to 1.1.0

**Also this notebook come with a post [Text Classification with XLNet in Action](https://medium.com/@yingbiao/text-classification-with-xlnet-in-action-869029246f7e)**<br>
**Feel free to check it, hope that it could help you.**

## Load data

**Load CSV data**

In [ ]:
data_path = "/content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/" 

In [ ]:
data_file_address = "/content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/News-TL.csv"

In [ ]:
df_data = pd.read_csv(data_file_address,sep=",",encoding="utf-8",names=['labels','texts'])

In [ ]:
df_data.columns

Index(['labels', 'texts'], dtype='object')

In [ ]:
df_data.head(n=20)

labels                                              texts
0        0  Memes Breaking: Hillary cancels public appeara...
1        0  Posted on October 27, 2016 by Pamela Geller \n...
2        0  Russian experts collecting evidence of anti-go...
3        0  A study conducted at QIMR Berghofer Medical Re...
4        0  Tomorrow, the U.S. election will take place, a...
5        0  Sniveling Cowards and NeverTrumpers Mitt Romne...
6        0  By Les Visible on November 4, 2016 Visible Ori...
7        0  Choose a topic Choose a topic All information,...
8        0  Breitbart ‚Äì by Deborah Danan \nTEL AVIV ‚Äì ...
9        0  40 Views November 09, 2016 GOLD , KWN King Wor...
10       0  By Amanda Froelich at trueactivist.com\nThe ne...
11       0  Russian experts collecting evidence of anti-go...
12       0  Email Print In every competition out there, th...
13       0  Insider Leaks Bill‚Äôs 2-Word Nickname For Hil...
14       0  TRUMPED! LIBERAL NEWS OUTLET BLOOMBERG POLL CO...
15       0    Ian Greenhalgh is a photographer and histori...
16       0  Home ‚Ä∫ HEALTH | US NEWS ‚Ä∫ CONTROVERSIAL NE...
17       0  Home ‚Ä∫ POLITICS | US NEWS ‚Ä∫ TRIGGERED: JOU...
18       0  Financial Markets , Market Manipulation , U.S....
19       0  Now that FBI‚Äôs reopened Hillary investigatio...

**Have a look labels**

In [ ]:
df_data.labels.unique()

array([0, 1])

In [ ]:
# Analyse the labels distribution
df_data.labels.value_counts()

1    3161
0    3117
Name: labels, dtype: int64

## Parser data

**Parser data into document structure**

In [ ]:
# Get sentence data
sentences = df_data.texts.to_list()
sentences[0]

'Memes Breaking: Hillary cancels public appearance due to a large crowd of people chanting "Lock Her Up!" The New York Times featured a photo of Hillary Clinton being welcomed to an early voting site in Pompano Beach, Florida on Sunday. Apparently, she didn\'t stay too long at her rally location at the Pompano Beach Amphitheater. Clinton almost melted at the sight of Trump signs surrounding her and people yelling "Lock Her Up!". A police escort shared what happened: Share on Facebook Posted Tuesday, November 01, 2016 Suggested Videos'

In [ ]:
# Get tag labels data
labels = df_data.labels.to_list()
print(labels[0])

0


**Make TAG name into index for training**

In [ ]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
# 0:negative, 1: positive
tag2idx={'0': 0,
 '1': 1}

In [ ]:
tag2idx

{'0': 0, '1': 1}

In [ ]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
tag2name

{0: '0', 1: '1'}

## Make tranning data

Make raw data into trainable data for XLNet, including:

- Set gpu environment
- Load tokenizer and tokenize
- Set 3 embedding, token embedding, mask word embedding, segmentation embedding
- Split data set into train and validate, then send them to dataloader

**Set up gpu environment**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
n_gpu

0

### Load tokenizer

Remember to install sentencepiece with  'pip install sentencepiece'

In [ ]:
# Manual define vocabulary address, if you download the model in local
# The vocabulary can download from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model"
vocabulary = '/content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/models/xlnet-base-cased/xlnet-base-cased-spiece.model'

In [ ]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 64

In [ ]:
!pip install sentencepiece

In [ ]:
# With cased model, set do_lower_case = False
tokenizer = XLNetTokenizer(vocab_file=vocabulary,do_lower_case=False)

### Set text input embedding

- token id embedding
- mask embedding
- segment embedding

The Embedding process was referred to [XLNet official repo](https://github.com/zihangdai/xlnet/blob/master/classifier_utils.py)


**This process is huge differnent from BERT**

In [ ]:
max_len  = 64

full_input_ids = []
full_input_masks = []
full_segment_ids = []

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

UNK_ID = tokenizer.encode("<unk>")[0]
CLS_ID = tokenizer.encode("<cls>")[0]
SEP_ID = tokenizer.encode("<sep>")[0]
MASK_ID = tokenizer.encode("<mask>")[0]
EOD_ID = tokenizer.encode("<eod>")[0]

for i,sentence in enumerate(sentences):
    # Tokenize sentence to token id list
    tokens_a = tokenizer.encode(sentence)
    
    # Trim the len of text
    if(len(tokens_a)>max_len-2):
        tokens_a = tokens_a[:max_len-2]
        
        
    tokens = []
    segment_ids = []
    
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(SEG_ID_A)
        
    # Add <sep> token 
    tokens.append(SEP_ID)
    segment_ids.append(SEG_ID_A)
    
    
    # Add <cls> token
    tokens.append(CLS_ID)
    segment_ids.append(SEG_ID_CLS)
    
    input_ids = tokens
    
    # The mask has 0 for real tokens and 1 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [0] * len(input_ids)

    # Zero-pad up to the sequence length at fornt
    if len(input_ids) < max_len:
        delta_len = max_len - len(input_ids)
        input_ids = [0] * delta_len + input_ids
        input_mask = [1] * delta_len + input_mask
        segment_ids = [SEG_ID_PAD] * delta_len + segment_ids

    assert len(input_ids) == max_len
    assert len(input_mask) == max_len
    assert len(segment_ids) == max_len
    
    full_input_ids.append(input_ids)
    full_input_masks.append(input_mask)
    full_segment_ids.append(segment_ids)
    
    if 3 > i:
        print("No.:%d"%(i))
        print("sentence: %s"%(sentence))
        print("input_ids:%s"%(input_ids))
        print("attention_masks:%s"%(input_mask))
        print("segment_ids:%s"%(segment_ids))
        print("\n")
    
    
    
    

No.:0
sentence: Memes Breaking: Hillary cancels public appearance due to a large crowd of people chanting "Lock Her Up!" The New York Times featured a photo of Hillary Clinton being welcomed to an early voting site in Pompano Beach, Florida on Sunday. Apparently, she didn't stay too long at her rally location at the Pompano Beach Amphitheater. Clinton almost melted at the sight of Trump signs surrounding her and people yelling "Lock Her Up!". A police escort shared what happened: Share on Facebook Posted Tuesday, November 01, 2016 Suggested Videos
input_ids:[1275, 4316, 17, 26834, 60, 9455, 8468, 23, 281, 2295, 542, 22, 24, 392, 2264, 20, 104, 21761, 17, 12, 1039, 6834, 727, 3539, 136, 12, 32, 158, 313, 1602, 2600, 24, 2532, 20, 9455, 1005, 163, 6442, 22, 48, 319, 3931, 335, 25, 23004, 3674, 2933, 19, 1442, 31, 485, 9, 14093, 19, 85, 314, 26, 46, 1078, 269, 206, 38, 7739, 7739]
attention_masks:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Set label embedding

In [ ]:
# Make label into id
tags = [tag2idx[str(lab)] for lab in labels]
print(tags[0])

0


## Split data into train and validate

70% for training, 30% for validation

**Split all data**

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(full_input_ids, tags,full_input_masks,full_segment_ids, 
                                                            random_state=4, test_size=0.3)

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(4394, 1884, 4394, 1884)

**Set data into tensor**

Not recommend tensor.to(device) at this process, since it will run out of GPU memory

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

**Put data into data loader**

In [ ]:
# Set batch num
batch_num = 32

In [ ]:
# Set token embedding, attention embedding, segment embedding
train_data = TensorDataset(tr_inputs, tr_masks,tr_segs, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks,val_segs, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

## Train model

**Load XLNet model**

In [ ]:
# In this document, contain confg(txt) and weight(bin) files
# The folder must contain: pytorch_model.bin, config.json
model_file_address = '/content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/models/xlnet-base-cased'

In [ ]:
# Will load config and weight with from_pretrained()
# Recommand download the model before using
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin"
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json" 
# ATTENTION!, rename "xlnet-base-cased-pytorch_model.bin" into "pytorch_model.bin"
# rename "xlnet-base-cased-config.json" inot "config.json"
model = XLNetForSequenceClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))

Some weights of the model checkpoint at /content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/models/xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/models/xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weigh

In [ ]:
model;

In [ ]:
# Set model to GPU,if you are using GPU machine
model.to(device);

In [ ]:
# Add multi GPU support
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [ ]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

### Set fine tuning method

**Manual optimizer**

In [ ]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
# Since XLNet in 'pytorch_transformer' did not contian classifier layers
# FULL_FINETUNING = True need to set True
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

### Fine-tuing model

In [ ]:
# TRAIN loop
model.train();

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs,b_labels = batch
        
        # forward pass
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        loss, logits = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
        

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 4394
  Batch size = 32
  Num steps = 690


Epoch:  20%|██        | 1/5 [59:32<3:58:08, 3572.19s/it]

Train loss: 0.25824744787311904


Epoch:  40%|████      | 2/5 [1:58:53<2:58:27, 3569.07s/it]

...

Train loss: 0.059800509936567124


Epoch:  80%|████████  | 4/5 [3:57:43<59:27, 3567.19s/it]  

Train loss: 0.03868749314693081


Epoch: 100%|██████████| 5/5 [4:57:04<00:00, 3564.83s/it]

Train loss: 0.02644710318333245


## Save model 

In [ ]:
xlnet_out_address = '/content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/models/xlnet_out_model/tc02'

In [ ]:
# Make dir if not exits
if not os.path.exists(xlnet_out_address):
        os.makedirs(xlnet_out_address)

In [ ]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [ ]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(xlnet_out_address, "pytorch_model.bin")
output_config_file = os.path.join(xlnet_out_address, "config.json")

In [ ]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(xlnet_out_address)

('/content/drive/My Drive/Colab Notebooks/Fake_News/FN-Transfer-Learning/models/xlnet_out_model/tc02/spiece.model',)

## Load model

In [ ]:
model = XLNetForSequenceClassification.from_pretrained(xlnet_out_address,num_labels=len(tag2idx))

In [ ]:
# Set model to GPU
model.to(device);

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

## Eval model

In [ ]:
# Evalue loop
model.eval();

In [ ]:
# Set acc funtion
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [ ]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_true = []
y_predict = []
print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs,b_labels = batch
    
    with torch.no_grad():
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]
    
    # Get textclassification predict result
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
#     print(tmp_eval_accuracy)
#     print(np.argmax(logits, axis=1))
#     print(label_ids)
    
    # Save predict and real label reuslt for analyze
    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
        
    for real_result in label_ids.tolist():
        y_true.append(real_result)

    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
   
    nb_eval_steps += 1
    
    
eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / len(val_inputs)
loss = tr_loss/nb_tr_steps 
result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'loss': loss}
report = classification_report(y_pred=np.array(y_predict),y_true=np.array(y_true))

# Save the report into file
output_eval_file = os.path.join(xlnet_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
        print("  %s = %s"%(key, str(result[key])))
        writer.write("%s = %s\n" % (key, str(result[key])))
        
    print(report)
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =1884
  Batch size = 32
***** Eval results *****
  eval_accuracy = 0.9660297239915074
  eval_loss = 0.21692988151853107
  loss = 0.02644710318333245
              precision    recall  f1-score   support

           0       0.99      0.94      0.96       924
           1       0.94      0.99      0.97       960

    accuracy                           0.97      1884
   macro avg       0.97      0.97      0.97      1884
weighted avg       0.97      0.97      0.97      1884

